In [ ]:
# Django notebook kernel must be running
# python manage.py shell_plus --notebook

In [ ]:
import django_initialiser
import time
import math
import numpy as np

In [ ]:
from api.models import Song
from api.melody_utils.generator import generate_all_piano_midi_de

In [ ]:
# generate_all_piano_midi_de("../midi/piano_midi_de")

In [ ]:
bach = Song.objects.filter(
    author__contains='bach'
)
elise = Song.objects.filter(
    title__contains='elise'
)
# print(bach[0].note_sequence)
print(list(map(int, elise[0].note_sequence.split(','))))

In [ ]:
# Melodies expected as lists
def longest_substring(song, query):
    # find the length of the strings
    m = len(song)
    n = len(query)

    # declaring the array for storing the dp values
    a = np.zeros(shape=[m + 1, n + 1], dtype='int')

    longest = 0

    # LCS DP
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                a[i][j] = 0

            elif song[i-1] == query[j-1]:
                a[i][j] = a[i-1][j-1] + 1
                longest = max(longest, a[i][j])

            else:
                a[i][j] = 0

    return longest

In [ ]:
def longest_common_subsequence(segment, query):
    # find the length of the strings
    m = len(segment)
    n = len(query)

    # declaring the array for storing the dp values
    a = np.zeros(shape=[m + 1, n + 1], dtype='int')

    longest = 0

    # LCS DP
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                a[i][j] = 0

            elif segment[i-1] == query[j-1]:
                a[i][j] = a[i-1][j-1] + 1

            else:
                a[i][j] = max(a[i-1][j], a[i][j-1])

    return a[m][n]

In [ ]:
# Break songs into segments of same length as query
def segmented_lookup(query):
    t = time.process_time()
    # Retrieve all songs from db
    songs = Song.objects.all()

    results = []
    query_len = len(query)

    for song in songs:
        song_notes = list(map(int, song.note_sequence.split(',')))
        
        song_len = len(song_notes)
        max_res = 0
        max_first_index = 0
        max_last_index = 0

        # Get segments
        for i in range(math.ceil(song_len / query_len)):
            first_note_index = i * query_len
            last_note_index = (i + 1) * query_len - 1

            # Last note index can be out of range but array slicing is safe against that
            res = longest_common_subsequence(song_notes[first_note_index:last_note_index], query)
            if res > max_res:
                max_res = res
                max_first_index = first_note_index
                max_last_index = last_note_index

        results.append((max_res, song, [max_first_index, max_last_index]))

    # Sort in descending order by the longest subsequence length
    results.sort(key=lambda tup: tup[0], reverse=True)
    t = time.process_time() - t
    print(f'Done in {t} seconds')
    return results


In [ ]:
# Old lookup function, compares query with entire song
def lookup(query):
    t = time.process_time()
    # Retrieve all songs from db
    songs = Song.objects.all()

    results = []

    for song in songs:
        song_notes = list(map(int, song.note_sequence.split(',')))
        res = longest_substring(song_notes, query)
        results.append((res, song))

    # Sort in descending order by the longest subsequence length
    results.sort(key=lambda tup: tup[0], reverse=True)
    t = time.process_time() - t
    print(f'Done in {t} seconds')
    return results

In [ ]:
input_search = [76, 75, 76, 75, 76, 71, 74, 72, 69, 57]
elise_notes = list(map(int, elise[0].note_sequence.split(',')))

In [ ]:
res = segmented_lookup(input_search)
for i in range(0, 3):
    print(res[i][0], res[i][1].title, f'at [{res[i][2][0]}, {res[i][2][1]}]')

In [ ]:
import threading
import queue

class QueueSearchTest:
    def __init__(self, query):
        self.query = query
        self.songs = Song.objects.all()
        self.results = []
        self.query_len = len(query)
        self.q = queue.Queue()
        self.threads = 6

    def worker(self):
        while True:
            song = self.q.get()

            song_notes = list(map(int, song.note_sequence.split(',')))
        
            song_len = len(song_notes)
            max_res = 0

            # Get segments
            for i in range(song_len - self.query_len + 1):
                first_note_index = i
                last_note_index = i + self.query_len - 1

                res = longest_common_subsequence(song_notes[first_note_index:last_note_index], self.query)
                max_res = max(max_res, res)

            self.results.append((max_res, song))

            self.q.task_done()

    def segmented_lookup(self):
        t = time.process_time()
        # Turn-on the worker threads.
        for i in range(self.threads):
            threading.Thread(target=self.worker, daemon=True).start()

        # Send all songs to the worker.
        for song in self.songs:
            self.q.put(song)

        # Block until all tasks are done.
        self.q.join()

        self.results.sort(key=lambda tup: tup[0], reverse=True)
        t = time.process_time() - t
        print(f'Done in {t} seconds')
        return self.results

In [ ]:
q = QueueSearchTest(input_search)
res = q.segmented_lookup()
for i in range(0, 3):
    print(res[i][0], res[i][1].title)